In the development of our chatbot application, we encountered a common challenge: the recurring expense of calling the OpenAI API every time a user poses a question. Despite the exceptional capabilities of OpenAI and other language processing services, the cost incurred by frequent API calls can become prohibitive, especially as the user base and question volume grow.

To address this challenge, we devised a solution that leverages the power of Redis cache coupled with a custom-built similarity model. Our objective was twofold: reduce costs associated with API calls and enhance response time for users.

Here's how our solution works:

1. **Cost Optimization**: Recognizing the financial implications of frequent API calls, we integrated Redis cache into our chatbot infrastructure. Redis serves as a repository where previously asked questions and their corresponding answers are stored. This way, instead of fetching responses from the API each time, we first check if a similar question has been posed before and retrieve the pre-computed answer from Redis, drastically reducing the need for expensive API calls.

2. **Enhanced Response Time**: Alongside the Redis cache implementation, we developed a custom similarity model utilizing nearest neighbor algorithms. This model analyzes incoming user questions and identifies the most similar questions from the dataset stored in Redis. By retrieving the best two similar questions from the cache, we expedite the process of finding relevant answers, further improving response time for users.

3. **Streamlined Workflow**: With our solution in place, the chatbot application now follows a streamlined workflow. Upon receiving a user question, the model first checks if similar questions exist in the Redis cache. If matches are found, it retrieves the corresponding answers directly from the cache, eliminating the need for costly API calls. In the absence of exact matches, the application seamlessly falls back to the OpenAI API, ensuring a comprehensive response strategy while minimizing expenses.

By implementing this innovative approach, we've not only mitigated the financial burden associated with API usage but also optimized the overall user experience by delivering prompt and accurate responses through intelligent caching and similarity analysis.

In [121]:
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from numpy import array
import numpy as np
# from sklearn.metrics.pairwise import cosine_similarity

In [122]:
# Sample Questions
questions = [
"What is the capital of France?",
"Where is the Louvre Museum located?",
"Where is the Eiffel Tower located?",
"Where can I find the Louvre Museum?",
"Which city is home to the Eiffel Tower?",
"What is the tallest structure in Paris?",
"Where can I see the Mona Lisa?",
"What city serves as the capital of France?",
"Which landmark is situated in Paris?",
"What is the main city in France?",
"Where is the famous glass pyramid located?",
"Where is the Seine River located?",
"What is the forecast for tomorrow?",
"Will it rain this weekend?",
"What are the latest smartphone features?",
"How does artificial intelligence impact daily life?",
"Who won the latest football match?",
"What are the rules of cricket?",
"What are the must-visit attractions in New York City?",
"How do I apply for a tourist visa?",
"What are some popular Italian dishes?",
"How do you make a classic spaghetti carbonara?",
"What are the symptoms of the common cold?",
"How can I improve my sleep quality?",
"What are the benefits of online learning?",
"How can I prepare for standardized tests effectively?",
"What events led to World War II?",
"Who were the key figures in the American Civil Rights Movement?",
"What are some tips for managing personal finances?",
"How does the stock market work?",
"What are the top-rated movies of all time?",
"Who won the latest Grammy Awards?",
"What is the history of the French Revolution?",
"Can you tell me about the tallest building in the world?",
"How tall is Burj Khalifa?",
"What is the population of Tokyo?",
"Where is the Great Wall of China located?",
"Can you explain the theory of relativity?",
"How deep is the Mariana Trench?",
"What is the capital of Brazil?",
"Where can I find the Statue of Liberty?",
"How trees make their food and create oxygen?",
"How many continents are there in the world?",
"What is the diameter of the Earth?",
"Which one is the most populous city in the USA?",
"Where is Mount Everest situated?",
"Can you explain the concept of natural selection?",
"How long is the Nile River?",
"What is the currency of Japan?",
"Where is the Colosseum located?",
"Where are the pyrimids located?",
"Can you define the term 'global warming'?",
"How many bones are there in the human body?",
"What is the main export of Australia?",
"Where is the Amazon Rainforest located?",
"Can you explain the process of cellular respiration?",
"How tall is the Empire State Building?",
"What is the capital of India?",
"Where is the Taj Mahal situated?",
"Can you describe the structure of an atom?",
"How many time zones are there in the world?",
"What is the largest desert in the world?",
"Where is the Vatican City located?",
"Can you explain the concept of gravity?",
"How many planets are there in the solar system?",
"What is the currency of the United Kingdom?",
"Where can I find the Mona Lisa?"
]


In [123]:
# Preprocessing (replace with your cleaning pipeline)
def preprocess(text):
    text = text.lower()  # Convert to lowercase
    text = "".join([char for char in text if char.isalnum() or char.isspace()])  # Remove punctuation
    # Initialize and train the Word2Vec model
    word2vec_model = Word2Vec([text.split()], min_count=1)
    return " ".join([word for word in text.split() if word in word2vec_model.wv.key_to_index.keys()])  # Filter out-of-vocabulary words


In [124]:
processed_questions = [preprocess(q) for q in questions]


In [125]:
# Feature Engineering (Word2Vec)
question_word_lists = [question.split() for question in processed_questions]
word2vec_model = Word2Vec(question_word_lists, min_count=1)
question_vectors = np.array([word2vec_model.wv[words].mean(axis=0) for words in question_word_lists])

In [126]:
# Option 2: Tf-idf
vectorizer = TfidfVectorizer()
question_vectors = vectorizer.fit_transform(processed_questions).toarray()

In [127]:
# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(question_vectors, np.arange(len(questions)), test_size=0.2)  # Dummy target (not used for KNN)


In [128]:
# KNN Model
model = NearestNeighbors(n_neighbors=2, algorithm='auto')
model.fit(X_train, y_train)


NearestNeighbors(n_neighbors=2)

In [129]:
# K-nearest neighbors classifier model
knn_model = KNeighborsClassifier(n_neighbors=2, metric='euclidean')
knn_model.fit(question_vectors, np.arange(len(questions)))  # Dummy labels


KNeighborsClassifier(metric='euclidean', n_neighbors=2)

In [130]:
# New Question Example
new_question = "Which country's currency is Yuan?"
new_question_processed = preprocess(new_question)
new_question_vector = vectorizer.transform([new_question_processed]).toarray()  # Convert new question to vector using Tf-idf

In [131]:
# Convert new question to vector using Tf-idf
knn_new_question_vector = vectorizer.transform([new_question_processed]).toarray()

# Reshape knn_new_question_vector (if necessary)
if knn_new_question_vector.shape[1] != question_vectors.shape[1]:  # Check for matching feature size
  knn_new_question_vector = knn_new_question_vector.reshape(1, -1)


In [132]:
# Find Similar Questions
distances, neighbors = model.kneighbors(new_question_vector)

# Check if the number of features matches the expected number
expected_num_features = len(knn_new_question_vector[0])  # Assuming knn_new_question_vector is a list of vectors
if knn_new_question_vector.shape[1] != expected_num_features:
    print("The number of features in knn_new_question_vector does not match the expected number of features.")
else:
    # Find Similar Questions
    knn_distances, knn_neighbors = knn_model.kneighbors(knn_new_question_vector)


In [133]:
print(distances)
print(neighbors)


[[1.02062876 1.14462216]]
[[46 52]]


In [134]:
print(knn_distances)
print(knn_neighbors)

[[1.02062876 1.10221298]]
[[48 65]]


In [135]:
# Print similar questions from training data based on their indices in neighbors
for i, neighbor_index in enumerate(neighbors[0]):
    print(f"Similar Question: {questions[neighbor_index]} (Distance: {distances[0][i]})")


Similar Question: Can you explain the concept of natural selection? (Distance: 1.020628759718866)
Similar Question: How many bones are there in the human body? (Distance: 1.144622161134244)


In [136]:
# Print similar questions from training data based on their indices in knn_neighbors
for i, knn_neighbor_index in enumerate(knn_neighbors[0]):
    print(f"Similar Question {i+1}: {questions[knn_neighbor_index]} (Distance: {knn_distances[0][i]})")

Similar Question 1: What is the currency of Japan? (Distance: 1.020628759718866)
Similar Question 2: What is the currency of the United Kingdom? (Distance: 1.1022129785319008)
